In [25]:
# Step 1: Preprocessing for CP-SAT Classroom Allocation

import pandas as pd

# Load your synthetic student dataset
import os
csv_path = 'synthetic_student_data_1000.csv'
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"File not found: {csv_path}. Please generate or copy the file to this directory.")
df = pd.read_csv(csv_path)  # Adjust path if needed

# --- Create Important Scores ---

# Reverse isolated and COVID so that higher score = better
df['Isolation_Score'] = 7 - df['isolated']
df['COVID_Score'] = 7 - df['COVID']

# Wellbeing Score (average of pwi_wellbeing, Isolation_Score, COVID_Score)
df['Wellbeing_Score'] = (df['pwi_wellbeing'] + df['Isolation_Score'] + df['COVID_Score']) / 3

# Bullying Score = criticises directly
df['Bullying_Score'] = df['criticises']

# Friendliness Score = School_support_engage
df['Friendliness_Score'] = df['School_support_engage']

# Academic Performance is already available as Academic_Performance

# Create a Student_ID field (if not already perfect)
df['Student_Index'] = range(len(df))

# --- Select only the needed columns for further processing ---
student_data = df[['Student_Index', 'Academic_Performance', 'Wellbeing_Score', 
                   'Bullying_Score', 'Friendliness_Score', 'language']]

# Show the first few rows
print(student_data.head())

# Check basic statistics
print("\nBasic Statistics:")
print(student_data.describe())


   Student_Index  Academic_Performance  Wellbeing_Score  Bullying_Score  \
0              0                    75         3.000000               1   
1              1                    68         2.666667               3   
2              2                    48         6.333333               1   
3              3                    71         3.000000               2   
4              4                    91         5.666667               6   

   Friendliness_Score  language  
0                5.25         0  
1                4.50         0  
2                4.75         0  
3                3.75         0  
4                4.50         0  

Basic Statistics:
       Student_Index  Academic_Performance  Wellbeing_Score  Bullying_Score  \
count    1000.000000           1000.000000      1000.000000     1000.000000   
mean      499.500000             70.107000         3.715000        4.034000   
std       288.819436             14.599682         1.377964        2.025573   
min       

In [26]:


# Step 2: Define Variables for CP-SAT Model
from ortools.sat.python import cp_model
import math

# Create a CP-SAT model
model = cp_model.CpModel()

# Define classroom size limit
class_size_limit = 25  # You can adjust if you want
num_students = len(student_data)  # Assuming student_data is defined elsewhere
num_classes = math.ceil(num_students / class_size_limit)

print(f"Total Students: {num_students}")
print(f"Total Classes Needed: {num_classes}")

# Create a variable for each student representing their assigned classroom
student_vars = []
for i in range(num_students):
    var = model.NewIntVar(0, num_classes - 1, f'student_{i}_class')
    student_vars.append(var)

print(f"Created {len(student_vars)} student variables.")

# Show first few variables info (example)
print(student_vars[:5])

Total Students: 1000
Total Classes Needed: 40
Created 1000 student variables.
[student_0_class(0..39), student_1_class(0..39), student_2_class(0..39), student_3_class(0..39), student_4_class(0..39)]


In [27]:
# Step 3: Adding Class Size Constraints

from collections import defaultdict

# Create a dictionary to group student variables by class
class_students = defaultdict(list)

for idx, var in enumerate(student_vars):
    for cls in range(num_classes):
        # Create an indicator variable: 1 if student idx is in class cls, 0 otherwise
        is_in_class = model.NewBoolVar(f'student_{idx}_is_in_class_{cls}')
        model.Add(var == cls).OnlyEnforceIf(is_in_class)
        model.Add(var != cls).OnlyEnforceIf(is_in_class.Not())
        class_students[cls].append(is_in_class)

# Allow class sizes to vary by at most 1 student for feasibility
min_class_size = num_students // num_classes
num_larger_classes = num_students % num_classes
max_class_size = min_class_size + 1 if num_larger_classes > 0 else min_class_size

for cls, students_in_class in class_students.items():
    if cls < num_larger_classes:
        model.Add(sum(students_in_class) == max_class_size)
    else:
        model.Add(sum(students_in_class) == min_class_size)

print(f"Added class size constraints for {num_classes} classes.")


Added class size constraints for 40 classes.


In [28]:
# Step 3B: Academic Performance Balance using total scores (simpler)

max_allowed_total_diff = 200  # Adjust as needed

class_total_scores = []
for cls in range(num_classes):
    # For each student, multiply their academic score by their assignment indicator
    total_score = model.NewIntVar(0, num_students * 100, f'class_{cls}_total_score')
    model.Add(total_score == sum(
        int(student_data.loc[idx, 'Academic_Performance']) * class_students[cls][idx]
        for idx in range(num_students)
    ))
    class_total_scores.append(total_score)

# Constrain the difference between any two classes' total scores
for i in range(num_classes):
    for j in range(i + 1, num_classes):
        diff = model.NewIntVar(0, num_students * 100, f'diff_{i}_{j}')
        model.AddAbsEquality(diff, class_total_scores[i] - class_total_scores[j])
        model.Add(diff <= max_allowed_total_diff)

print("Academic Performance Balance constraints (total scores) added between classes.")


Academic Performance Balance constraints (total scores) added between classes.


In [29]:
# Step 4: Calculate and summarize constraints for classroom allocation

# Calculate actual class sizes after solving (example, here just show intended sizes)
intended_class_sizes = [len(class_students[cls]) for cls in range(num_classes)]
print("Intended class sizes (number of students per class):", intended_class_sizes)

# Show the max and min intended class sizes
print("Max intended class size:", max(intended_class_sizes))
print("Min intended class size:", min(intended_class_sizes))

# Show the academic performance constraint used
print(f"Max allowed average academic difference between classes: {max_allowed_difference}")

# If you have solved the model, you can extract and print actual assignments and statistics here.

Intended class sizes (number of students per class): [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Max intended class size: 1000
Min intended class size: 1000
Max allowed average academic difference between classes: 10


In [30]:
# Step 5: Solve the model and extract assignments and statistics
from ortools.sat.python import cp_model

# Create a solver and solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    print('Solution found!')
    # Extract student assignments
    assignments = [solver.Value(var) for var in student_vars]
    student_data['Assigned_Class'] = assignments

    # Calculate actual class sizes
    actual_class_sizes = student_data.groupby('Assigned_Class').size().tolist()
    print('Actual class sizes:', actual_class_sizes)

    # Calculate average academic performance per class
    avg_academic_per_class = student_data.groupby('Assigned_Class')['Academic_Performance'].mean()
    print('Average academic performance per class:')
    print(avg_academic_per_class)

    # Calculate max difference in average academic performance
    max_diff = avg_academic_per_class.max() - avg_academic_per_class.min()
    print(f'Max difference in average academic performance: {max_diff:.2f}')

else:
    print('No solution found.')

Solution found!
Actual class sizes: [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25]
Average academic performance per class:
Assigned_Class
0     69.32
1     69.64
2     69.04
3     69.20
4     69.52
5     70.60
6     71.56
7     70.76
8     70.60
9     69.76
10    70.92
11    69.96
12    69.76
13    69.52
14    70.24
15    69.16
16    69.12
17    71.04
18    69.80
19    70.48
20    70.16
21    71.16
22    69.88
23    70.72
24    70.44
25    69.92
26    69.60
27    70.24
28    70.16
29    69.56
30    71.20
31    70.32
32    70.16
33    71.20
34    70.28
35    70.00
36    69.56
37    70.48
38    68.92
39    70.32
Name: Academic_Performance, dtype: float64
Max difference in average academic performance: 2.64


C:\Users\bilal\AppData\Local\Temp\ipykernel_13464\1268202414.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student_data['Assigned_Class'] = assignments


In [31]:
# Minimal working example: Only class size constraints
# Comment out Step 3B (academic constraints) above and rerun this cell.

from ortools.sat.python import cp_model

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    print('Solution found!')
    assignments = [solver.Value(var) for var in student_vars]
    student_data['Assigned_Class'] = assignments
    print(student_data['Assigned_Class'].value_counts())
else:
    print('No solution found. Try relaxing constraints further.')

Solution found!
Assigned_Class
26    25
8     25
15    25
11    25
24    25
12    25
28    25
10    25
3     25
14    25
30    25
31    25
17    25
20    25
9     25
6     25
4     25
25    25
33    25
13    25
21    25
36    25
35    25
27    25
39    25
37    25
19    25
34    25
38    25
1     25
2     25
22    25
32    25
0     25
5     25
29    25
23    25
7     25
18    25
16    25
Name: count, dtype: int64


C:\Users\bilal\AppData\Local\Temp\ipykernel_13464\3535995608.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student_data['Assigned_Class'] = assignments
